In [2]:
%pip install -q --upgrade rouge-score
%pip install -q --upgrade keras-nlp
%pip install -q --upgrade keras  # Upgrade to Keras 3.

#from ann_visualizer.visualize import ann_viz

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import keras_nlp
import pathlib
import random
import os
# Loss and metrics
import tensorflow as tf
from tensorflow import keras

os.chdir("/media/damian/PDB_DB/TRANSLATION/MODEL")
os.environ['CUDA_VISIBLE_DEVICES'] = "0"


import keras
from keras import ops

import tensorflow.data as tf_data
from tensorflow_text.tools.wordpiece_vocab import (
    bert_vocab_from_dataset as bert_vocab,
)


In [13]:
# Define constants
BATCH_SIZE = 256
EPOCHS = 300
MAX_SEQUENCE_LENGTH = 300
AA_VOCAB_SIZE = 100
PF_VOCAB_SIZE = 100
EMBED_DIM = 128
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

In [14]:
text_file = "GLOBAL_150_300_subset.csv"

In [15]:
def wordmaker_kmer(seq):
    k = 1 # length of the k-mer
    output = [seq[i:i+k] for i in range(len(seq) - k + 1)]
    return " ".join(output)

def wordmaker_non_overlapping(seq, word_length=1):
    output = [seq[i:i+word_length] for i in range(0, len(seq), word_length)]
    return " ".join(output)



In [16]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    AA, PF = line.split(",")
    AA = wordmaker_non_overlapping(AA.lower())
    PF = wordmaker_non_overlapping(PF)
    text_pairs.append((AA, PF))


In [17]:
for _ in range(5):
    print(random.choice(text_pairs))


('m r v l g v d p g l t r c g l g v v d g g l g i r a h l v e v g v v r t p a t a e v a e r l c a v s d g i d a w l d r t r p e a v a v e k v f s q a n m r t v m g t a q a g a v a i v l a a r r g l p v g l y t p s e v k a a v t g s g r a d k a q v s f m i t r l l g l t e a p r p a d a a d a l a l a l c h l w r g p a l a r f r s a a p g g p t r', 'u o j g b e e j j m o l g d d c h j n r t B I x A p o o o p l m k l p q z B D A z x x t o o q o k n p n n p r p q u v t t q r p l i k m r x E E K H G D D y u u q m q p k m q o n q t r q u y x y t u r p q p r s s p l m o m m p s t s y w y w w s n n p n k n p p p t v E A t s v t r o f h g a a d c c f i j m p q r w x x y C E F I L N S V U W Z Z')
('m l y p t p i a k l i d s f s k l p g i g a k t a t r l a f y t i s m s d e d v n d f a k n l l a a k r e l t y c s v c g r l t d d d p c i i c t d e t r d r t k i l v v e d s k d v s a m e k i q e y r g l y h v l q g l i s p m n g v g p d d i n l k s l i t r l m d s e v d e v i i a t n a t a d g e a t

In [18]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")


162998 total pairs
114100 training pairs
24449 validation pairs
24449 test pairs


In [19]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf_data.Dataset.from_tensor_slices(text_samples)
    vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(1000).prefetch(2),
        vocabulary_size=vocab_size,
        reserved_tokens=reserved_tokens,
    )
    return vocab



In [20]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]


AA_samples = [text_pair[0] for text_pair in train_pairs]
AA_vocab = train_word_piece(AA_samples, AA_VOCAB_SIZE, reserved_tokens)

PF_samples = [text_pair[1] for text_pair in train_pairs]
PF_vocab = train_word_piece(PF_samples, PF_VOCAB_SIZE, reserved_tokens)


2024-08-19 15:27:09.837864: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-19 15:27:13.578975: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [303]:
print("AA Tokens: ", AA_vocab[10:20])
print("Proflex Tokens: ", PF_vocab[10:20])

AA Tokens:  ['h', 'i', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's']
Proflex Tokens:  ['G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']


In [21]:
AA_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=AA_vocab, lowercase=False
)
PF_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=PF_vocab)


In [22]:
AA_input_ex = text_pairs[0][0]
AA_tokens_ex = AA_tokenizer.tokenize(AA_input_ex)
print("AA seq: ", AA_input_ex)
print("Tokens: ", AA_tokens_ex)
print(
    "Recovered seq after detokenizing: ",
    AA_tokenizer.detokenize(AA_tokens_ex),
)

print()

PF_input_ex = text_pairs[0][1]
PF_tokens_ex = PF_tokenizer.tokenize(PF_input_ex)
print("PF seq: ", PF_input_ex)
print("Tokens: ", PF_tokens_ex)
print(
    "Recovered seq after detokenizing: ",
    PF_tokenizer.detokenize(PF_tokens_ex),
)


AA seq:  m a s r g v n k v i l v g n l g q d p e v r y m p s g g a v a n l t l a t s e s w r d k q t g e m k e q t e w h r v v m f g k l a e v a g e y l r k g s q v y i e g q l r t r k w t d q s g q e r y t t e i n v p q i g g v m q m l g g r q g g g a p a g g q q q g g w g q p q q p q q p q g g n q f s g g a q s r p q q s a p a p s n e p p m d f d d d i p f
Tokens:  tf.Tensor(
[14  4 19 18  9 21 15 12 21 11 13 21  9 15 13  9 17  6 16  7 21 18 23 14
 16 19  9  9  4 21  4 15 13 20 13  4 20 19  7 19 22 18  6 12 17 20  9  7
 14 12  7 17 20  7 22 10 18 21 21 14  8  9 12 13  4  7 21  4  9  7 23 13
 18 12  9 19 17 21 23 11  7  9 17 13 18 20 18 12 22 20  6 17 19  9 17  7
 18 23 20 20  7 11 15 21 16 17 11  9  9 21 14 17 14 13  9  9 18 17  9  9
  9  4 16  4  9  9 17 17 17  9  9 22  9 17 16 17 17 16 17 17 16 17  9  9
 15 17  8 19  9  9  4 17 19 18 16 17 17 19  4 16  4 16 19 15  7 16 16 14
  6  8  6  6  6 11 16  8], shape=(176,), dtype=int32)
Recovered seq after detokenizing:  tf.Tensor(b'm a s r

In [23]:
def preprocess_batch(AA, PF):
    batch_size = ops.shape(PF)[0]

    AA = AA_tokenizer(AA)
    PF = PF_tokenizer(PF)

    # Pad `eng` to `MAX_SEQUENCE_LENGTH`.
    AA_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        pad_value=AA_tokenizer.token_to_id("[PAD]"),
    )
    AA = AA_start_end_packer(AA)

    # Add special tokens (`"[START]"` and `"[END]"`) to PF and pad it as well.
    PF_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=PF_tokenizer.token_to_id("[START]"),
        end_value=PF_tokenizer.token_to_id("[END]"),
        pad_value=PF_tokenizer.token_to_id("[PAD]"),
    )
    PF = PF_start_end_packer(PF)

    return (
        {
            "encoder_inputs": AA,
            "decoder_inputs": PF[:, :-1],
        },
        PF[:, 1:],
    )


def make_dataset(pairs):
    AA_texts, PF_texts = zip(*pairs)
    AA_texts = list(AA_texts)
    PF_texts = list(PF_texts)
    dataset = tf_data.Dataset.from_tensor_slices((AA_texts, PF_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.shuffle(1024).prefetch(64).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)


In [24]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")


inputs["encoder_inputs"].shape: (256, 300)
inputs["decoder_inputs"].shape: (256, 300)
targets.shape: (256, 300)


2024-08-19 15:27:45.261344: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-08-19 15:27:45.262205: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [26]:
# Encoder
encoder_inputs = keras.Input(shape=(MAX_SEQUENCE_LENGTH,), name="encoder_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=AA_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(encoder_inputs)

encoder_outputs = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)

encoder = keras.Model(encoder_inputs, encoder_outputs)

# Decoder
decoder_inputs = keras.Input(shape=(MAX_SEQUENCE_LENGTH,), name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(MAX_SEQUENCE_LENGTH, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=PF_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(decoder_inputs)

x = keras_nlp.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.1)(x)
decoder_outputs = keras.layers.Dense(PF_VOCAB_SIZE, activation="softmax")(x)

decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)

# Early stopping callback
early_stopping_callback = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=8,
    restore_best_weights=True
)

# Custom loss function
def custom_loss(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

# Learning rate schedule
learning_rate = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,  # Lower initial learning rate
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True
)

# Optimizer
optimiser = keras.optimizers.Adam(learning_rate)

transformer.summary()
# Compile the model
transformer.compile(optimizer=optimiser, loss=custom_loss, metrics=['accuracy'])

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_and_position… │ (None, 300, 128)  │     51,200 │ encoder_inputs[0… │
│ (TokenAndPositionE… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 300, 128)  │    593,024 │ token_and_positi… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_7        │ (None, 300, 100)  │    723,428 │ decoder_inputs[0… │
│ (Functional)        │                   │            │ transformer_enco… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,367,652 (5.22 MB)

 Trainable params: 1,367,652 (5.22 MB)

 Non-trainable params: 0 (0.00 B)

In [309]:
transformer.summary()
# Compile the model
transformer.compile(optimizer=optimiser, loss=custom_loss, metrics=['accuracy'])

# Train the model
history = transformer.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[early_stopping_callback]
)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_and_position… │ (None, 300, 128)  │     51,200 │ encoder_inputs[0… │
│ (TokenAndPositionE… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 300, 128)  │    593,024 │ token_and_positi… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_33       │ (None, 300, 100)  │    723,428 │ decoder_inputs[0… │
│ (Functional)        │                   │            │ transformer_enco… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,367,652 (5.22 MB)

 Trainable params: 1,367,652 (5.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
446/446 ━━━━━━━━━━━━━━━━━━━━ 3973s 9s/step - accuracy: 0.0539 - loss: 3.4806 - val_accuracy: 0.1371 - val_loss: 3.1701
Epoch 2/300
446/446 ━━━━━━━━━━━━━━━━━━━━ 3956s 9s/step - accuracy: 0.5997 - loss: 0.7495 - val_accuracy: 0.7413 - val_loss: 0.0041
Epoch 3/300
446/446 ━━━━━━━━━━━━━━━━━━━━ 3952s 9s/step - accuracy: 0.7432 - loss: 0.0041 - val_accuracy: 0.7413 - val_loss: 0.0029
Epoch 4/300
446/446 ━━━━━━━━━━━━━━━━━━━━ 3970s 9s/step - accuracy: 0.7432 - loss: 0.0030 - val_accuracy: 0.7413 - val_loss: 0.0025
Epoch 5/300
446/446 ━━━━━━━━━━━━━━━━━━━━ 3974s 9s/step - accuracy: 0.7432 - loss: 0.0026 - val_accuracy: 0.7413 - val_loss: 0.0023
Epoch 6/300
446/446 ━━━━━━━━━━━━━━━━━━━━ 3973s 9s/step - accuracy: 0.7432 - loss: 0.0024 - val_accuracy: 0.7413 - val_loss: 0.0022
Epoch 7/300
446/446 ━━━━━━━━━━━━━━━━━━━━ 3962s 9s/step - accuracy: 0.7433 - loss: 0.0023 - val_accuracy: 0.7413 - val_loss: 0.0022
Epoch 8/300
446/446 ━━━━━━━━━━━━━━━━━━━━ 3959s 9s/step - accuracy: 0.6497 - loss: 0

In [34]:
from tensorflow.keras.models import load_model

custom_objects = {
    'custom_loss': custom_loss,
    # Add other custom objects here if needed
}

model_path = 'model2.keras'
transformer = load_model(model_path, custom_objects=custom_objects)
#transformer.compile(optimizer=optimiser, loss=custom_loss, metrics=['accuracy'])


#%pip install graphviz
#import graphviz
#plot_model(transformer, to_file='transformer_model.png', show_shapes=True, show_layer_names=True)

In [39]:
import tensorflow as tf
import random


def decode_sequences(input_sentences):
    batch_size = 1
    
    # Tokenize the encoder input 
    encoder_input_tokens = AA_tokenizer(input_sentences)
    if isinstance(encoder_input_tokens, tf.RaggedTensor):
        encoder_input_tokens = encoder_input_tokens.to_tensor()
    encoder_input_tokens = tf.convert_to_tensor(encoder_input_tokens, dtype=tf.int32)
    
    # Limit input sequence
    input_length = tf.shape(encoder_input_tokens)[1]
    if input_length > MAX_SEQUENCE_LENGTH:
        raise ValueError("Input sequence length exceeds MAX_SEQUENCE_LENGTH.")
    
    # Pad the input sequence 
    if input_length < MAX_SEQUENCE_LENGTH:
        pads = tf.fill([batch_size, MAX_SEQUENCE_LENGTH - input_length], 0)
        encoder_input_tokens = tf.concat([encoder_input_tokens, pads], axis=1)
    
    def next(prompt, cache, index):
        logits = transformer([encoder_input_tokens, prompt])[:, index - 1, :]
        hidden_states = None
        return logits, hidden_states, cache

    # Build a prompt of length MAX_SEQUENCE_LENGTH with a start token and padding tokens.
    start = tf.fill([batch_size, 1], PF_tokenizer.token_to_id("[START]"))
    pad = tf.fill([batch_size, MAX_SEQUENCE_LENGTH - 1], PF_tokenizer.token_to_id("[PAD]"))
    prompt = tf.concat([start, pad], axis=1)

    # Greedy sampling
    generated_tokens = keras_nlp.samplers.GreedySampler()(
        next,
        prompt,
        stop_token_ids=[PF_tokenizer.token_to_id("[END]")],
        index=1,  # Start sampling after start token.
    )
    
    # Truncate or pad the generated tokens to ensure they match the input length
    if tf.shape(generated_tokens)[1] > input_length:
        generated_tokens = generated_tokens[:, :input_length]
    elif tf.shape(generated_tokens)[1] < input_length:
        pads = tf.fill([batch_size, input_length - tf.shape(generated_tokens)[1]], PF_tokenizer.token_to_id("[PAD]"))
        generated_tokens = tf.concat([generated_tokens, pads], axis=1)
    
    # Detokenize the generated tokens to obtain the output sentences.
    generated_sentences = PF_tokenizer.detokenize(generated_tokens)
    
    return generated_sentences

# Apply decoder
test_AA = [pair[0] for pair in test_pairs if len(pair[0]) <= MAX_SEQUENCE_LENGTH]

for i in range(1):
    seq = random.choice(test_AA)
    translated = decode_sequences([seq])
    translated = translated.numpy()[0].decode("utf-8")
    translated = (
        translated.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )
    print(f'{seq}')
    print(f'{translated}')
    print()


m s t l e q k l t e m l t a p v e a l g f e l v g i e f i r g r t s t l r i y i d s e d g i n v d d c a d v s h q v s a v m d v e d p i t v a y n l e v s s p g l d r p m f t a e h y q r f t g e e v a l v l r m a v q n r r k w q g i i k a v d g e m i t v t v e g k d e v f a l s n i q k a n l v p h f
P Q J D N I J Q D K J Q A M R D A J F E D J R F H D E H O F O Q P Q J O H T H C P D C F H L R C C B A C R P G N R P A R K C R D C M H Q R A T L J D R P P M F J C O M K E Q A D G T N O E Q F D D R A J R J O K A R N L O O I S N F H H I A R C F D K H Q R Q R D F I C D R E A J P L H N I A L J R M G E



In [310]:
# save model

transformer.save('model2.keras')
